In [1]:
import dotenv

dotenv.load_dotenv()

True

In [2]:
from hospital_review_chain import (
    reviews_vector_chain
)

h:\Documents\I Ingenieur Troisième année\Cours\z Autre\Projet LLM\Hospital_LLM_RAG\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Test hospital review chain

In [4]:
query = """ What have patients said about hospital efficiency?
            Mention details from specific reviews."""
response = reviews_vector_chain.invoke(query)
print(response)

{'query': ' What have patients said about hospital efficiency?\n            Mention details from specific reviews.', 'result': 'One patient, Megan Allison, described the staff at Rose Inc as "efficient." No other reviews mentioned hospital or staff efficiency. \n'}


In [6]:
query = """ What have patients said about hospital cleanliness?"""
response = reviews_vector_chain.invoke(query)
print(response["result"])

Here's a summary of patient feedback on hospital cleanliness:

* **Schultz-Powers:** Bethany Farrell was "pleasantly surprised" by the cleanliness of the facilities and praised the janitorial staff for maintaining a hygienic environment. 

* **Vaughn PLC:** Donald Sparks's stay was negatively impacted by a "lack of cleanliness" and he stressed the importance of hygiene for preventing infections. Lorraine Scott, however, said she "appreciate[d] the hospital's effort to maintain a clean environment" and commended the janitorial staff.

* **Pearson LLC:** Monique Foster found the hospital's cleanliness and hygiene standards "commendable." 



In [7]:
query = """ What have patients said about hospital staff?"""
response = reviews_vector_chain.invoke(query)
print(response["result"])

Here's a summary of patient comments regarding hospital staff:

* **Burke, Griffin and Cooper:** Patients described the staff as "compassionate."
* **Malone, Thompson and Mejia:** Patients found the staff to be "friendly and attentive."
* **Boyd PLC:** Patients reported that the staff were "friendly and caring." 



# Test Hospital Cypher Chains

In [1]:
import dotenv

dotenv.load_dotenv()

True

In [4]:
from hospital_cypher_chain import (
    hospital_cypher_chain
)

h:\Documents\I Ingenieur Troisième année\Cours\z Autre\Projet LLM\Hospital_LLM_RAG\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
question = """What is the average visit duration for
              emergency visits in North Carolina?"""
response = hospital_cypher_chain.invoke(question)
print(response["result"])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (v:Visit)-[:AT]->(h:Hospital)
WHERE v.admission_type = 'Emergency' AND h.state_name = 'NC' AND v.discharge_date IS NOT NULL
WITH v.admission_date AS admission, v.discharge_date AS discharge
RETURN avg(duration.inDays(date(admission), date(discharge)).days) AS average_duration

Full Context:
[{'average_duration': 15.833783783783785}]

> Finished chain.
The average visit duration for emergency visits in North Carolina is 15.83 days. 



In [12]:
question = """Which state had the largest percent increase
              in Medicaid visits from 2022 to 2023?
              Use 'Medicaid as the payer name'"""
response_2 = hospital_cypher_chain.invoke(question)
response_2["result"]



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (h:Hospital)<-[:AT]-(v:Visit)-[:COVERED_BY]->(p:Payer)
WHERE p.name = 'Medicaid' AND v.admission_date >= '2022-01-01' AND v.admission_date < '2024-01-01'
WITH h.state_name AS state, COUNT(v) AS visit_count,
     SUM(CASE WHEN v.admission_date >= '2022-01-01' AND v.admission_date < '2023-01-01' THEN 1 ELSE 0 END) AS count_2022,
     SUM(CASE WHEN v.admission_date >= '2023-01-01' AND v.admission_date < '2024-01-01' THEN 1 ELSE 0 END) AS count_2023
WITH state, visit_count, count_2022, count_2023,
     (toFloat(count_2023 - count_2022)) / toFloat(count_2022) * 100 AS percent_increase
RETURN state, percent_increase
ORDER BY percent_increase DESC
LIMIT 1

Full Context:
[{'state': 'TX', 'percent_increase': 8.823529411764707}]

> Finished chain.


'Texas had an 8.82% increase in Medicaid visits from 2022 to 2023. \n'